## Lets "pivot" data and nest it into a list.
We currently have our tracks_by_album table modeled as such:

````
CREATE TABLE music.tracks_by_album (
    album_title text,
    album_year int,
    track_number int,
    album_genre text static,
    performer text static,
    track_title text,
    PRIMARY KEY ((album_title, album_year), track_number)
````

Let's model it as one partition per album, and the track titles in a list


In [ ]:
%%Cql create table if not exists music.albums (album_title text,
album_genre text,
album_year int,
performer text,
tracks list<text>, 
primary key ((album_title, album_year)) )

In [ ]:
%%Cql truncate music.albums

### Now a handy case class

In [ ]:
case class Track(album_title: String,
album_year:Int,
track_number:Int,
album_genre: Option[String],
performer: Option[String],
track_title: String)

### And an RDD to pick up the original data

In [ ]:
val tracks = sc.cassandraTable[Track]("music","tracks_by_album")

In [ ]:
tracks.first

## Create a Pair RDD with key, track name.  The key in this case (album_title, year).  So we want tuples ((album_title, year), track_title).  Also filter out tracks that are null

In [ ]:
val track_pairs = tracks.filter(t => t.track_title != null).
map(t => ((t.album_title, t.album_year), t.track_title))

In [ ]:
track_pairs.first

## Use groupByKey, to convert it to (key, collection of track names)

In [ ]:
val albums_with_tracks = track_pairs.groupByKey

In [ ]:
albums_with_tracks.first


## Flatten it out so we have (album_title, year, collection of tracks)

In [ ]:
val albums_with_tracks2 = albums_with_tracks.
map{ case ((album, year), tracks) => (album, year, tracks) }

In [ ]:
albums_with_tracks2.first.toString

In [ ]:
albums_with_tracks2.saveToCassandra("music","albums",
            SomeColumns("album_title", "album_year", "tracks"))

## ... And check it out

In [ ]:
%%Cql select * from music.albums limit 5;

## Now we need to fill in genre and year.  Let's just upsert them in!  This will also insert any albums with no tracks

In [ ]:
tracks.saveToCassandra("music","albums",
SomeColumns("album_title","album_year","album_genre","performer"))

In [ ]:
%%Cql select * from music.albums limit 5;